In [5]:
import openai
import os
import pandas as pd
text_file = open("API_key.txt", "r")

#read whole file to a string
openai.api_key =  text_file.read()

#close file
text_file.close()



In [17]:
text_file = open("API_key.txt", "r")

#read whole file to a string
openai.api_key =  text_file.read()

#close file
text_file.close()


split_files = []

def split_file_by_character(filename, character):
    with open(filename, 'r') as f:
        lines = f.readlines()

    start_line = 0
    prev_end_line = -1
    for i, line in enumerate(lines):
        if character in line:
            prev_end_line = i
            extracted_lines = lines[start_line:prev_end_line]

            # join the extracted lines into a string
            extracted_text = "".join(extracted_lines)
            if extracted_text != "":
                split_files.append([filename,(start_line,prev_end_line),extracted_text])
            #print(filename)
            start_line = i 
    
    extracted_lines = lines[prev_end_line+1:len(lines)]
    # join the extracted lines into a string
    extracted_text = "".join(extracted_lines)
    split_files.append([filename,(prev_end_line+1,len(lines)),extracted_text])
    
    return 


file_paths_details = [
    "index.html",
    "style.css",
    "login-page.js"
]

comment_chars = [
    "<!-",
    "/*",
    "//"
]

for i in range(0,len(file_paths_details)):
        parts = split_file_by_character(file_paths_details[i],comment_chars[i])
        

print("Total number of functions extracted:", len(split_files))

df = pd.DataFrame(split_files)
df.columns = ["filepath","BlockStartStop","Code"]
display(df)

Total number of functions extracted: 10


,filepath,BlockStartStop,Code
0,index.html,"(0, 3)","<!DOCTYPE html>\n<html lang=""en"">\n\n"
1,index.html,"(3, 14)",<!- File References ->\n\n<head>\n <meta char...
2,index.html,"(14, 22)","<!- Login Page Holder ->\n<main id=""main-holde..."
3,index.html,"(23, 32)","<form id=""login-form"">\n <input type=""tex..."
4,style.css,"(0, 4)",/* Page Styles */\nhtml {\n height: 100%;\n}\n
5,style.css,"(4, 15)",/* Body Styles */\nbody {\n height: 100%;\n ...
6,style.css,"(15, 27)",/* Holder for login items*/\n#main-holder {\n ...
7,style.css,"(27, 53)",/* Login Error Message Styles */\n#login-erro...
8,style.css,"(54, 85)",#login-form {\n align-self: flex-start;\n di...
9,login-page.js,"(1, 17)","const loginForm = document.getElementById(""log..."


In [9]:
def get_embedding(task):
    response = openai.Embedding.create(
            input=task,
            model="text-embedding-ada-002"
        )
    return response['data'][0]['embedding']

In [25]:
df['code_embedding'] = df['Code'].apply(lambda x: get_embedding(x))
df.to_csv("code_search_openai-python.csv", index=False)

In [11]:

df = pd.read_csv('code_search_line_embeddings.csv')

df['code_embedding'] = df.code_embedding.apply(lambda x: [float(y) for y in x[1:-1].split(",")])

In [22]:
display(df)

,filepath,Code,LineNumber,similarities,code_embedding
0,index.html,<!DOCTYPE html>\n,0,0.753720,"[-0.01256885752081871, 0.0028874874114990234, ..."
1,index.html,"<html lang=""en"">\n",1,0.742165,"[-0.009006879292428493, 0.010268648155033588, ..."
2,index.html,\n,2,0.752977,"[0.002454084809869528, -0.018236806616187096, ..."
3,index.html,<!- File References ->\n,3,0.751432,"[-0.025017328560352325, -0.0020156055688858032..."
4,index.html,\n,4,0.753019,"[0.0024455352686345577, -0.01827205717563629, ..."
...,...,...,...,...,...
129,login-page.js,location.reload();\n,12,0.722429,"[-0.018045900389552116, -0.0164205189794302, -..."
130,login-page.js,} else {\n,13,0.684728,"[-0.03090381622314453, -0.012458906508982182, ..."
131,login-page.js,loginErrorMsg.style.opacity = 1;\n,14,0.747025,"[-0.007993947714567184, -0.014100635424256325,..."
132,login-page.js,}\n,15,0.710626,"[-0.019705967977643013, -0.017968423664569855,..."


In [16]:
import numpy 
import matplotlib
task = 'Add this image in the login page holder https://blog.ipleaders.in/wp-content/uploads/2019/11/foodmitho.jpg'
from openai.embeddings_utils import cosine_similarity

def search_functions(df, code_query):
    embedding = get_embedding(code_query)
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = df.sort_values('similarities', ascending=False).head(1)
    return res

res = search_functions(df, task)

display(df)

print(res.iloc[0]['Code'])

KeyboardInterrupt: 

In [39]:
def replace(filename, startStop, new_code_block):
    startStop = eval(startStop)
    # Read in the contents of the file
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Delete the lines between start_index and stop_index
    del lines[startStop[0]:startStop[1]+1]

    # Insert the new_code_block at start_index
    lines[startStop[0]:startStop[0]] = new_code_block

    # Write the modified contents back to the file
    with open(filename, 'w') as file:
        file.writelines(lines)


In [40]:
n = "<!DOCTYPE html>\n<html lang="
import fileinput
replace("test.txt", "[0,1]",n)